# To Do
* Check for commas in the data in query and replace all with a special character
* More efficient get_columns() call

# Imports

In [2]:
import pickle
import pandas as pd
import datetime
from sqlalchemy import create_engine
import time
from collections import Counter

# Functions

In [3]:
'''Specific data queries'''

def get_columns(schema, table):
    # Start a timer
    start = time.time()
    
    query = pd.read_sql_query('SELECT TOP 1 * FROM '+schema+"."+table+';', ENGINE)
    return query.columns, "\n Time it took in seconds: " + str(time.time() - start)

def get_data(schema, table, columns, start_date, end_date, filters, groupings=False):
    start = time.time()
    
    params_list = []
    params_list.append(start_date)
    params_list.append(end_date)
    
    columns_as_string = ', '.join(columns)
    
    filters_query_list = []
    
    for i in filters:
        filters_query_list.append(' AND '+i+' = %s')
        params_list.append(str(filters[i]))
    
    filters_query_string = "".join(filters_query_list)
    
    print(columns_as_string)
    print(params_list)
    
    query = pd.read_sql_query(
        'SELECT '+columns_as_string+' FROM '+schema+"."+table+
        ' WHERE report_date BETWEEN %s AND %s'+filters_query_string+';', ENGINE, params=params_list)
    
    return query, "\n Time it took in seconds: " + str(time.time() - start)
    

def get_nm_test2(schema, table, start_date, end_date):
    '''Creates the Pkl file.
    Schema should be a string of the schema desired'''

    # Start a timer
    start = time.time()

    # Run the query to get all contents from schema
    query = pd.read_sql_query(
        'SELECT medium, source, campaign, adcontent, keyword, usertype, devicecategory, '+
        'pagepath_clean, pageviews, timeonpage, uniquepageviews, entrances, exits, bounces, report_date '+
        'FROM '+schema+"."+table+
        ' WHERE report_date BETWEEN %s AND %s AND application_display_name = %s;',
        ENGINE, params=[
            start_date, end_date,
            'X']
        )

    return query, "\n Time it took in seconds: " + str(time.time() - start)

def get_schemas():
    start = time.time()
    schemas = pd.read_sql_query('select nspname from pg_namespace', ENGINE)
    non_temp_schemas = schemas[~schemas['nspname'].str.contains('temp')]['nspname']
    return non_temp_schemas, "\n Time it took in seconds: " + str(time.time() - start)

def care_and_conditions_grouping_function(data):
    # Create group category names based on logic
    grouping_levels = ['first level grouping', 'second level grouping']

    start = time.time()
    clean_stop_pagepath_clean = data['pagepath_clean'].str.split(pat=r'[?#\., \[\]\']')
    split_pagepath_clean = clean_stop_pagepath_clean.str[0].str.split(pat=r'/')
    
    for i in range(len(grouping_levels)):
        data[grouping_levels[i]] = split_pagepath_clean.str[i+1].str.replace('-', ' ').str.title()

        df_counter = pd.DataFrame.from_dict(Counter(data[grouping_levels[i]]), orient='index').reset_index()
        df_low = df_counter[df_counter[0] < 10]

        data.loc[data[grouping_levels[i]].isin(df_low['index']), grouping_levels[i]] = "Other"
        
    data = data[data[grouping_levels[0]] == "Conditions And Care Areas"]
    data = data.fillna(value="None")

    return data, "\n Time it took in seconds: " + str(time.time() - start)

# User Input Section

## Define Server Connection

In [1]:
# User input
SERVER = input("Server: ")
DATABASE = input("Database: ")
USERNAME = input("Username: ")
PASSWORD = input("Password: ")
PORT = input("Port: ")

In [4]:
ENGINE = create_engine(
    'postgresql://'+USERNAME+':'+PASSWORD+'@'+SERVER+':'+PORT+'/'+DATABASE)

## Select Schema and Table

In [5]:
# Getting a list of schemas in the database

schemas, schema_time = get_schemas()
print(schemas.values)

In [29]:
# User input schema
# Select one of the schemas above

SCHEMA = input("Enter the schema desired: ")

In [30]:
# Getting list of tables for a particular schema

available_tables = pd.read_sql_query(
    'select table_name from X where table_schema = %s;',
        ENGINE, params=[
            SCHEMA])
print(available_tables)

In [32]:
# USER INPUT
# Select which table you want access to
TABLE = input("Enter the table desired: ")

## Select Filters, Groupings, Columns, Date Range, Update Daily

In [33]:
# TODO: find a more efficient way to get the column names
# Right now, calling the first row and reading the columns instead of reading directly
columns, columns_time = get_columns(SCHEMA, TABLE)
print(columns)
print(columns_time)

In [34]:
# Define empty column and dict
grouping_column_list = []
filters_dict = {}

In [35]:
# FILTERS HERE
# These are chosen by user
# Select a column to filter on, then select what columns you want to include

# Filter by application display name mandatory, otherwise there is duplicate data
filters_dict[
    'application_display_name'] = 'X'

In [36]:
# GROUPINGS HERE
# These are defined by us, need to request one for a client
# User would decide on what to apply here, but they are not actually applied until after the data is pulled down

# Columns that the user selects to be grouped on must be included in the initial query, but do not necessarily
#need to be included in the final export. If they did not indicate keeping the columns, they can be deleted
#from the dataframe before converting to a csv.

care_and_conditions_grouping = True

if care_and_conditions_grouping:
    grouping_column_list.append('pagepath_clean')

In [37]:
# USER INPUT
# Select columns to interact with on the screen, data to pull in
# Don't need to select columns just because we use them in where

COLUMNS_FOR_TABLEAU = [
    'medium', 'source', 'campaign', 'usertype', 'devicecategory', 'pagepath_clean', 'pageviews', 'report_date', 'timeonpage']

# column_list needs to be a set becuase of the potential to add the same column many time for different requirement
columns_list = set(grouping_column_list + COLUMNS_FOR_TABLEAU)
START_DATE = datetime.date(2016, 8, 21)
END_DATE = datetime.date(2017, 12, 31) # Possibly left blank to go to today

UPDATE_DAILY = False # Only available if END_DATE is left blank

## Run Queries and Collect Data

In [38]:
data, data_time = get_data(SCHEMA, TABLE, columns_list, START_DATE, END_DATE, filters_dict)
print(data)
print(data_time)

In [21]:
data['pageviews'].sum()

## Do Custom Groupings

In [53]:
# If the grouping was selected above, do this function
if care_and_conditions_grouping:
    data, care_and_conditions_grouping_time = care_and_conditions_grouping_function(data)
    

In [34]:
print(data.head())

## Export to CSV

In [35]:
# Consolidate columns for export so it only contains those desired
data = data.drop(list(set(grouping_column_list) - set(COLUMNS_FOR_TABLEAU)), axis=1)

# Export to csv
start_csv_export = time.time()
data.to_csv('tableau_example_extract.csv', index=False)
time_csv_export = time.time() - start_csv_export
print(time_csv_export)